In [1]:
## โค้ดนี้พัฒนาจาก A-aggValuePM3-alertDisplay-withTrigger.ipynb

In [2]:
## โจทย์ทางธุรกิจ คือ ธุรกิจต้องการรับการแจ้งเตือนเมื่อมีค่าฝุ่น (pm 2.5) เกินกว่าค่า threshold 
## โดยให้มีการคำนวณเปรียบเทียบกับ threshold ในทุกๆ 5 วินาทีที่ processing platform (Spark) ได้รับข้อมูล (Message Arrival)

In [3]:
# จำเป็นต้องเปลี่ยนเลขไอพีของ Kafka broker โดยใช้ INTERNAL IP Address พร้อมด้วยหมายเลข port 9092

kafka_broker = "10.128.0.12:9092"

In [4]:
# สร้าง Spark session โดยใช้การตั้งค่าที่กำหนด

from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("KafkaSubscribe").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "1000m").\
        config("spark.executor.cores", "2").\
        config("spark.cores.max", "6").\
        config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0").\
        getOrCreate()



Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b8df3ea7-5f68-4a32-bacf-f1c34d302b6f;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.0 in central
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.4-3 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.7.5 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 652ms :: artifacts dl 13

# 1. ได้รับ Unbounded input table 

In [5]:
# สร้าง DataFrame โดยอ่านข้อมูลจาก Kafka stream

unboundInput_table_df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", kafka_broker) \
  .option("subscribe", "quickstart-events") \
  .option("group.id", "Aekanun-Spark-App") \
  .load()

In [6]:
from pyspark.sql import functions as sparkf

In [7]:
# แสดง schema ของ column ต่างๆ ใน DataFrame

unboundInput_table_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



# 2. เกิดการ Query - Stream Data Processing

In [8]:
#extractedDateLatLong_df.filter(sparkf.col('isMobile').isNotNull())

# 3. ได้รับ Result table

In [9]:
result_table_df = unboundInput_table_df

In [10]:
# แสดงรายชื่อ column เป็น list

result_table_df.columns

['key', 'value', 'topic', 'partition', 'offset', 'timestamp', 'timestampType']

In [11]:
result_table_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



# 4. ได้รับ Output (table)

In [ ]:
from pyspark.sql import functions as F



output_df = result_table_df.writeStream \
    .format("console") \
    .outputMode("update") \
    .trigger(processingTime='5 seconds') \
    .start()

output_df.awaitTermination()


23/06/22 00:31:38 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-1fea67e6-092c-48a7-bc68-9ff4ec3196dc. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+---+-----+-----+---------+------+---------+-------------+
|key|value|topic|partition|offset|timestamp|timestampType|
+---+-----+-----+---------+------+---------+-------------+
+---+-----+-----+---------+------+---------+-------------+



-------------------------------------------
Batch: 1
-------------------------------------------
+----+--------------------+-----------------+---------+------+--------------------+-------------+
| key|               value|            topic|partition|offset|           timestamp|timestampType|
+----+--------------------+-----------------+---------+------+--------------------+-------------+
|null|[7B 22 6C 6F 63 6...|quickstart-events|        0|     0|2023-06-22 00:31:...|            0|
|null|[7B 22 6C 6F 63 6...|quickstart-events|        0|     1|2023-06-22 00:31:...|            0|
|null|[7B 22 6C 6F 63 6...|quickstart-events|        0|     2|2023-06-22 00:31:...|            0|
|null|[7B 22 6C 6F 63 6...|quickstart-events|        0|     3|2023-06-22 00:31:...|            0|
|null|[7B 22 6C 6F 63 6...|quickstart-events|        0|     4|2023-06-22 00:31:...|            0|
|null|[7B 22 6C 6F 63 6...|quickstart-events|        0|     5|2023-06-22 00:31:...|            0|
|null|[7B 22 6C 6F 63

23/06/22 00:31:55 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 5257 milliseconds


-------------------------------------------
Batch: 2
-------------------------------------------
+----+--------------------+-----------------+---------+------+--------------------+-------------+
| key|               value|            topic|partition|offset|           timestamp|timestampType|
+----+--------------------+-----------------+---------+------+--------------------+-------------+
|null|[7B 22 6C 6F 63 6...|quickstart-events|        0|  1480|2023-06-22 00:31:...|            0|
|null|[7B 22 6C 6F 63 6...|quickstart-events|        0|  1481|2023-06-22 00:31:...|            0|
|null|[7B 22 6C 6F 63 6...|quickstart-events|        0|  1482|2023-06-22 00:31:...|            0|
|null|[7B 22 6C 6F 63 6...|quickstart-events|        0|  1483|2023-06-22 00:31:...|            0|
|null|[7B 22 6C 6F 63 6...|quickstart-events|        0|  1484|2023-06-22 00:31:...|            0|
|null|[7B 22 6C 6F 63 6...|quickstart-events|        0|  1485|2023-06-22 00:31:...|            0|
|null|[7B 22 6C 6F 63

-------------------------------------------
Batch: 3
-------------------------------------------
+----+--------------------+-----------------+---------+------+--------------------+-------------+
| key|               value|            topic|partition|offset|           timestamp|timestampType|
+----+--------------------+-----------------+---------+------+--------------------+-------------+
|null|[7B 22 6C 6F 63 6...|quickstart-events|        0|  3960|2023-06-22 00:31:...|            0|
|null|[7B 22 6C 6F 63 6...|quickstart-events|        0|  3961|2023-06-22 00:31:...|            0|
|null|[7B 22 6C 6F 63 6...|quickstart-events|        0|  3962|2023-06-22 00:31:...|            0|
|null|[7B 22 6C 6F 63 6...|quickstart-events|        0|  3963|2023-06-22 00:31:...|            0|
|null|[7B 22 6C 6F 63 6...|quickstart-events|        0|  3964|2023-06-22 00:31:...|            0|
|null|[7B 22 6C 6F 63 6...|quickstart-events|        0|  3965|2023-06-22 00:31:...|            0|
|null|[7B 22 6C 6F 63

In [ ]:
output_df.stop()